In [114]:
from mxnet import nd 
import random 
import zipfile

In [115]:
with zipfile.ZipFile('F:/深度学习文件/书籍/动手学深度学习/data/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars = f.read().decode('utf-8')
print(corpus_chars[:48])
len(corpus_chars)

想要有直升机
想要和你飞到宇宙去
想要和你融化在一起
融化在宇宙里
我每天每天每天在想想想想著你


63282

In [116]:
corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
corpus_chars_data = corpus_chars[0: 60000]
corpus_chars_data[:20]

'想要有直升机 想要和你飞到宇宙去 想要和'

In [117]:
# 建⽴字符索引
idx_to_char = list(set(corpus_chars_data))
char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)]) # 枚举变量，依次赋值
vocab_size = len(char_to_idx)
vocab_size, idx_to_char

(2537,
 ['宙',
  '威',
  '挂',
  '闻',
  '娥',
  '珀',
  '嘟',
  '两',
  '他',
  '陽',
  '边',
  '室',
  '腐',
  '侦',
  '君',
  '阱',
  '知',
  '射',
  '扬',
  '焰',
  '收',
  '琐',
  '忆',
  '搶',
  '耍',
  '呢',
  '生',
  '缀',
  '稚',
  '泥',
  '往',
  '盈',
  '落',
  '建',
  '坚',
  '阶',
  '叹',
  '憔',
  '倭',
  '湛',
  '角',
  '雏',
  '热',
  '薄',
  '昵',
  '穿',
  '摔',
  '躬',
  '箏',
  '蔽',
  '快',
  '氛',
  '铅',
  '邪',
  '误',
  '巴',
  '燒',
  '孝',
  '嫦',
  '匙',
  '邮',
  '议',
  '捏',
  '纪',
  '标',
  '胚',
  '教',
  '贝',
  '憶',
  '熟',
  '铁',
  '及',
  '走',
  '瀰',
  '师',
  'Y',
  '席',
  '抹',
  '狰',
  '遐',
  '怎',
  '砍',
  'B',
  '挖',
  '題',
  '辗',
  '稀',
  '穗',
  '哟',
  '麼',
  '裡',
  '柜',
  '恶',
  '忍',
  '们',
  '挡',
  '躯',
  '话',
  '哮',
  '震',
  '诀',
  '深',
  '曹',
  '辛',
  '悴',
  '喬',
  '问',
  '慢',
  '佻',
  '摩',
  '揍',
  '丹',
  '幕',
  '巾',
  '青',
  '脏',
  '满',
  '旦',
  '晶',
  '赶',
  '秒',
  '完',
  '伏',
  '地',
  '啥',
  '迫',
  '梦',
  '频',
  '眸',
  '輕',
  '剔',
  '诞',
  '到',
  '正',
  '船',
  '嚣',
  '程',
  '宠',
  '妹',
  '猪',
  'd',
  '机',

In [118]:
corpus_indices = [char_to_idx[char] for char in corpus_chars_data]
sample = corpus_indices[: 20]
print(sample)

[1049, 411, 2449, 1216, 2529, 141, 1489, 1049, 411, 1488, 2026, 1918, 132, 1714, 0, 270, 1489, 1049, 411, 1488]


In [119]:
# 随机采样
def data_iter_random(corpus_indices, batch_size, num_steps, ctx=None):
    num_examples = (len(corpus_indices) - 1) // num_steps
    epoch_size = num_examples // batch_size
    example_indices = list(range(num_examples))
    random.shuffle(example_indices)
    
    def _data(pos):
        return corpus_indices[pos: pos + num_steps]
    
    for i in range(epoch_size):
        i = i * batch_size
        batch_indices = example_indices[i: i + batch_size]
        X = [_data(j * num_steps) for j in batch_indices]
        Y = [_data(j * num_steps + 1) for j in batch_indices]
        yield nd.array(X, ctx), nd.array(Y, ctx)

In [120]:
my_seq = list(range(30))
for X, Y in data_iter_random(my_seq, 2, 6):
    print(X, Y)



[[ 6.  7.  8.  9. 10. 11.]
 [ 0.  1.  2.  3.  4.  5.]]
<NDArray 2x6 @cpu(0)> 
[[ 7.  8.  9. 10. 11. 12.]
 [ 1.  2.  3.  4.  5.  6.]]
<NDArray 2x6 @cpu(0)>

[[18. 19. 20. 21. 22. 23.]
 [12. 13. 14. 15. 16. 17.]]
<NDArray 2x6 @cpu(0)> 
[[19. 20. 21. 22. 23. 24.]
 [13. 14. 15. 16. 17. 18.]]
<NDArray 2x6 @cpu(0)>


In [121]:
def data_iter_consecutive(corpus_indices, batch_size, num_steps, ctx=None):
    corpus_indices = nd.array(corpus_indices, ctx=ctx)
    data_len = len(corpus_indices)
    batch_len = data_len // batch_size
    indices = corpus_indices[0: batch_size * batch_len].reshape((batch_size, batch_len))
    epoch_size = (batch_len - 1) // num_steps  # 
    print(indices)
    for i in range(epoch_size):
        i  = i * num_steps
        X = indices[:, i : i + num_steps]
        Y = indices[:, i + 1: i + num_steps]
        yield X, Y

In [122]:
for X, Y in data_iter_consecutive(my_seq, batch_size=2, num_steps=6):
    print(X, Y)


[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14.]
 [15. 16. 17. 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29.]]
<NDArray 2x15 @cpu(0)>

[[ 0.  1.  2.  3.  4.  5.]
 [15. 16. 17. 18. 19. 20.]]
<NDArray 2x6 @cpu(0)> 
[[ 1.  2.  3.  4.  5.]
 [16. 17. 18. 19. 20.]]
<NDArray 2x5 @cpu(0)>

[[ 6.  7.  8.  9. 10. 11.]
 [21. 22. 23. 24. 25. 26.]]
<NDArray 2x6 @cpu(0)> 
[[ 7.  8.  9. 10. 11.]
 [22. 23. 24. 25. 26.]]
<NDArray 2x5 @cpu(0)>


In [123]:
nd.one_hot(nd.array([0, 2]), vocab_size)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x2537 @cpu(0)>

In [124]:
import d2lzh as d2l
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

In [125]:
len(corpus_indices), vocab_size, char_to_idx, idx_to_char

(60000,
 2537,
 {'宙': 0,
  '威': 1,
  '挂': 2,
  '闻': 3,
  '娥': 4,
  '珀': 5,
  '嘟': 6,
  '两': 7,
  '他': 8,
  '陽': 9,
  '边': 10,
  '室': 11,
  '腐': 12,
  '侦': 13,
  '君': 14,
  '阱': 15,
  '知': 16,
  '射': 17,
  '扬': 18,
  '焰': 19,
  '收': 20,
  '琐': 21,
  '忆': 22,
  '搶': 23,
  '耍': 24,
  '呢': 25,
  '生': 26,
  '缀': 27,
  '稚': 28,
  '泥': 29,
  '往': 30,
  '盈': 31,
  '落': 32,
  '建': 33,
  '坚': 34,
  '阶': 35,
  '叹': 36,
  '憔': 37,
  '倭': 38,
  '湛': 39,
  '角': 40,
  '雏': 41,
  '热': 42,
  '薄': 43,
  '昵': 44,
  '穿': 45,
  '摔': 46,
  '躬': 47,
  '箏': 48,
  '蔽': 49,
  '快': 50,
  '氛': 51,
  '铅': 52,
  '邪': 53,
  '误': 54,
  '巴': 55,
  '燒': 56,
  '孝': 57,
  '嫦': 58,
  '匙': 59,
  '邮': 60,
  '议': 61,
  '捏': 62,
  '纪': 63,
  '标': 64,
  '胚': 65,
  '教': 66,
  '贝': 67,
  '憶': 68,
  '熟': 69,
  '铁': 70,
  '及': 71,
  '走': 72,
  '瀰': 73,
  '师': 74,
  'Y': 75,
  '席': 76,
  '抹': 77,
  '狰': 78,
  '遐': 79,
  '怎': 80,
  '砍': 81,
  'B': 82,
  '挖': 83,
  '題': 84,
  '辗': 85,
  '稀': 86,
  '穗': 87,
  '哟': 88,
  '麼': 89,
  '裡'

In [126]:
def to_onehot(X, size):
    return [nd.one_hot(x, size) for x in X.T]

In [127]:
to_onehot(X, vocab_size)[0]


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<NDArray 2x2537 @cpu(0)>

In [128]:
# 初始化模型参数
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()

In [129]:
print('will use ', ctx)

will use  cpu(0)


In [130]:
def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    
    # 隐含层参数
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    
    # 输出层参数
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    
    # 附上梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params


In [131]:
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [132]:
def rnn(inputs, state, params):
# inputs和outputs皆为num_steps个形状为(batch_size, vocab_size)的矩阵
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [133]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(6, (2, 2537), (2, 256))

In [134]:
# 定义预测函数
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state, 
               num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        # 将上⼀时间步的输出作为当前时间步的输⼊
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
        # 计算输出和更新隐藏状态
        (Y, state) = rnn(X, state, params)
        # 下⼀个时间步的输⼊是prefix⾥的字符或者当前的最佳预测字符
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [135]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
                ctx, idx_to_char, char_to_idx)

'分开纱擎羞伦魏意哈忽办腳'

In [136]:
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [137]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                            vocab_size, ctx, corpus_indices, idx_to_char,
                            char_to_idx, is_random_iter, num_epochs, num_steps,
                            lr, clipping_theta, batch_size, pred_period,
                            pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()
    for epoch in range(num_epochs):
        if not is_random_iter: # 如使⽤相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter: # 如使⽤随机采样，在每个⼩批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else: # 否则需要使⽤detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
                (outputs, state) = rnn(inputs, state, params)
                # 拼接之后形状为(num_steps * batch_size, vocab_size)
                outputs = nd.concat(*outputs, dim=0)
                # Y的形状是(batch_size, num_steps)，转置后再变成⻓度为
                # batch * num_steps 的向量，这样跟输出的⾏⼀⼀对应
                y = Y.T.reshape((-1,))
                # 使⽤交叉熵损失计算平均分类误差
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx) # 裁剪梯度
            d2l.sgd(params, lr, 1) # 因为误差已经取过均值，梯度不⽤再做平均
            l_sum += l.asscalar() * y.size
            n += y.size
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                        prefix, pred_len, rnn, params, init_rnn_state,
                        num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

In [138]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [141]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                        vocab_size, ctx, corpus_indices, idx_to_char,
                        char_to_idx, True, num_epochs, num_steps, lr,
                        clipping_theta, batch_size, pred_period, pred_len,
                        prefixes)

epoch 50, perplexity 40.399811, time 26.96 sec
 - 分开 我只是这样的小丑 我会不能你我已经 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我
 - 不分开  是我说不见你 说你的眼里 消灭                                 
epoch 100, perplexity 14.659497, time 26.30 sec
 - 分开始重痛 想这个时间的风傲 看我还爱你 我不多 我们我在我手 我们的天 再了我 就只有这样 我感着你的
 - 不分开了一定 等待 我有第打有一天 我 在我爱上睡   用                        
epoch 150, perplexity 8.940916, time 26.46 sec
 - 分开始重深　 你给我若多　　 我的本笑的你好有 载不再这么简我而的爱 你就会听我的热把我 经你的依尚车默
 - 不分开 我要的你爱你会有了 说就是一场旅现 在江外的城 有一种味道叫做家 陆羽泡的茶 有幅泼墨 我有爱你 
epoch 200, perplexity 6.409320, time 27.47 sec
 - 分开始重新 不要在我的爱 有谁在练烧 来静的美诞    我看你那的二 走让我们在山服  吹一页 梦故的岁
 - 不分开 我很悔你心碎没人帮你擦眼泪 别离开身边拥大你我的爱情才就记   不过过悉 你无久了我 你不要在我 
epoch 250, perplexity 4.975427, time 26.65 sec
 - 分开始重新 我在你一边画着 你不想你 嘴坏不知 你让我不懂不见 你的改变不同 所有的人们在远远的想说 你
 - 不分开了 我说好有种样 不知不觉 我已经这节牵 后知后觉 后知后觉 却在那场 在小叮 的表好只剩下一种 等


In [142]:
# 相邻法训练模型
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                        vocab_size, ctx, corpus_indices, idx_to_char,
                        char_to_idx, False, num_epochs, num_steps, lr,
                        clipping_theta, batch_size, pred_period, pred_len,
                        prefixes)

epoch 50, perplexity 37.078063, time 26.52 sec
 - 分开了  你就会的承  在你的等 有一种味 叫到你看的感   我 想 哎哟的山 你 如果 oh的 旧黏呢
 - 不分开  我 说我们的每情 一天 训壮忿 你的眼 有你 我的灵觉 我忘不是 不是再 说 哎哟啦哟的美   
epoch 100, perplexity 17.181010, time 26.54 sec
 - 分开 你说的爱 有一个一的姿道 我说的爱好  我有了你想笑 你我会 你的终 你给  你一句其特  你在我
 - 不分开 寒想这里 想是我不能够 我知道了我手 而你的微里 我在远 是我有的梦 我亲不会 手哈编看 我一种身
epoch 150, perplexity 12.002001, time 26.61 sec
 - 分开了吧 我都能够承着我獨的  再手会功听我 我不是要不到你 我知道你手都没有帮 只是你的地 这样味墨的
 - 不分开 寒只在很天 不过的影好   其他了部                               


KeyboardInterrupt: 